### Extract IRIS SJI cubes into single FITS-images
This is a preparation step in order to load IRIS SJI into JHV. It does not contain downloading the original IRIS files.

In [2]:
import os
import pyvo  # !pip install pyvo
import numpy as np
from sunpy.map import Map
from irisreader import observation

In [3]:
# get observations and iris annotations via TAP, select those with many spectra that were classified as supersonic downflows
serv = pyvo.dal.TAPService("https://tap.cs.technik.fhnw.ch/tap")  # May 2022 "crobs4"

join = "iris_obs.epn_core as a inner join iris_obs.cr_events as b on a.granule_uid = b.obs_id"
fields = "a.obs_id,a.raster_lines,a.sji_lines,b.supersonic_downflows" # ,a.hek_url
filt = "supersonic_downflows > 999 and a.raster_lines LIKE '%Si IV 1394%'"
r = serv.run_sync(f"select {fields} from {join} where {filt} ORDER BY supersonic_downflows DESC")

r

<Table length=201>
          obs_id           ... supersonic_downflows
          object           ...        int32        
-------------------------- ... --------------------
20150408_003144_3800600036 ...                14200
20150813_113412_3600600039 ...                14097
20150815_103942_3600600039 ...                12187
20130830_232946_4000257165 ...                 9836
20150814_113012_3600600039 ...                 8472
20150627_072544_3625581135 ...                 8290
20131023_142944_3823251483 ...                 8021
20150307_121344_3800600036 ...                 7839
20141204_181904_3800100083 ...                 7836
20160128_115744_3680600932 ...                 6731
                       ... ...                  ...
20150410_195243_3800100082 ...                 1081
20150529_075615_3820256897 ...                 1078
20150108_022532_3823250065 ...                 1074
20160121_123744_3680100932 ...                 1074
20150422_041924_3823601388 ...               

In [ ]:
%%time
# extract top50 and save to NAS - this can take a while (50 obs took 1:42:10)

# settings
base_folder = "/nas08-data02/silvan/cr_obs3"
quantile = 99.9  # 99.9 -> ~1350px, 99.99 -> ~140px
sji_line_gamma = {
    'C II 1330': [0.4],
    'Si IV 1400': [0.4],
    'Mg II h&k 2796': [0.4, 1],
    'Mg II wing 2832': [1]
}

""" 1. iterate through (top n) results and read obs """
for ob in range(0,50):
    obsid = r[ob]['obs_id']
    if type(obsid) == bytes:
        obsid = obsid.decode('utf-8') # decode obsid-string if necessary - astropy votable seems to use bytes..
    if not os.path.exists(f"/data2/iris/{obsid[0:4]}/{obsid[4:6]}/{obsid[6:8]}/{obsid}/"):
        print(f"skipped obs: {obsid} (data after 2015 currently unavailable)")
        continue
    obs = observation(f"/data2/iris/{obsid[0:4]}/{obsid[4:6]}/{obsid[6:8]}/{obsid}/") # read obs
    
    """ 2. iterate through SJI-lines of current obs """
    sji_line_desc = [line.replace("/","&") for line in obs.sji.get_lines()['description']]  # replace / with & to avoid 2 nested folders
    for idx_sji in range(obs.n_sji):
        n = len(obs.sji[idx_sji].headers)
        
        """ 3. get quantile of all images in this line """
        exptimes = obs.sji[idx_sji].get_exptimes()
        data_max = 0  # max
        for i in range(n):  # find data_max
            data_max = max(data_max, np.percentile(obs.sji[idx_sji].get_image_step(i), quantile) / exptimes[i])

        """ 4. select gamma correction - for line 2796 we generate 2 versions """
        for gamma in sji_line_gamma[sji_line_desc[idx_sji]]:
            folder = f"{base_folder}/{obsid}/{sji_line_desc[idx_sji]}"
            if gamma == 1:  # len(ji_line_gamma[sji_line_desc[idx_sji]]) > 1 and 
                folder += "_linear"  # add linear term to folder
            
            """ 5. create output folder - skip if already exists """
            if os.path.exists(folder):
                print(f"skipped line: {folder} (already exists)")
                continue
            os.makedirs(folder)

            """ 6. save single images with in JHV-friendly format. Value 0 is used for invalid (=transparent) pixels """
            for i in range(n):
                h = obs.sji[idx_sji].headers[i]
                h['HV_DMIN'] = 0  # instead of DATAMIN/MAX
                h['BLANK'] = 0
                data = obs.sji[idx_sji].get_image_step(i).copy()
                idxtbl = data==-200  # invalid pixels
                data /= exptimes[i]
                h['HV_DMAX'] = data_max
                data = data.clip(h['HV_DMIN'], h['HV_DMAX'])   #  + (h['HV_DMAX']/255)
                data = data**gamma  # gamma correction
                h['HV_DMAX'] = h['HV_DMAX']**gamma  # gamma correction
                data = ((data - h['HV_DMIN']) / h['HV_DMAX'] * 254).astype('uint8') + 1
                data[idxtbl] = 0

                Map(data, h).save(f"{folder}/{i}.fits")  # save
        